## Access Vessel Underway Csiro (Parquet)
This Jupyter notebook demonstrates how to access and plot vessel_underway_csiro data, available as a [Parquet](https://parquet.apache.org) dataset stored on S3.

🔗 More information about the dataset is available [in the AODN metadata catalogue](https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/catalog.search#/metadata/None).

📌 The source of truth for this notebook is maintained on [GitHub](https://github.com/aodn/aodn_cloud_optimised/tree/main/notebooks/vessel_underway_csiro.ipynb).


In [1]:
dataset_name = "vessel_underway_csiro"

## Install/Update packages and Load common functions

In [ ]:
import os, requests, importlib.util

open('setup.py', 'w').write(requests.get('https://raw.githubusercontent.com/aodn/aodn_cloud_optimised/main/notebooks/setup.py').text)

spec = importlib.util.spec_from_file_location("setup", "setup.py")
setup = importlib.util.module_from_spec(spec)
spec.loader.exec_module(setup)

setup.install_requirements()
setup.load_dataquery()

In [2]:
from DataQuery import GetAodn

# Understanding the Dataset

## Understanding Parquet Partitioning

Parquet files can be **partitioned** by one or more columns, which means the data is physically organised into folders based on the values in those columns. This is similar to how databases use indexes to optimise query performance.

Partitioning enables **faster filtering**: when you query data using a partitioned column, only the relevant subset of files needs to be read—improving performance significantly.

For example, if a dataset is partitioned by `"site_code"`, `"timestamp"`, and `"polygon"`, filtering on `"site_code"` allows the system to skip unrelated files entirely.

In this notebook, the `GetAodn` class includes built-in methods to efficiently filter data by **time** and **latitude/longitude** using the **timestamp** and **polygon** partitions. Other partitions can be used for filtering via the `scalar_filter`.

Any filtering on columns that are **not** partitioned can be significantly slower, as all files may need to be scanned. However, the `GetAodn` class provides a `scalar_filter` method that lets you apply these filters at load time—before the data is fully read—helping reduce the size of the resulting DataFrame.

Once the dataset is loaded, further filtering using Pandas is efficient and flexible.

See further below in the notebook for examples of how to filter the data effectively.

To view the actual partition columns for this dataset, run:


In [3]:
aodn = GetAodn(bucket_name="aodn-cloud-optimised", prefix="", s3_fs_opts={ "client_kwargs": {"endpoint_url": "http://127.0.0.1:9000"}, "key": "minioadmin", "secret":"minioadmin"})
dname = f'{dataset_name}.parquet'
%time aodn_dataset = aodn.get_dataset(dname)

CPU times: user 25.1 ms, sys: 3.67 ms, total: 28.8 ms
Wall time: 8.05 s


### aodn_dataset.dataset.partitioning.schema

## List unique partition values

In [4]:
%%time
unique_partition_value = aodn_dataset.get_unique_partition_values('voyage')
print(list(unique_partition_value)[0:2])  # showing a subset only

['in2023_v05', 'in2019_v01']
CPU times: user 470 µs, sys: 0 ns, total: 470 µs
Wall time: 397 µs


## Visualise Spatial Extent of the dataset
This section plots the polygons representing the areas where data is available. It helps to identify and create a bounding box around the regions containing data.

In [5]:
aodn_dataset.plot_spatial_extent()

/home/lbesnard/miniforge3/envs/AodnCloudOptimised/lib/python3.12/site-packages/cartopy/mpl/feature_artist.py:144: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '


## Get Temporal Extent of the dataset

Similary to the spatial extent, we're retrieving the minimum and maximum timestamp partition values of the dataset. This is not necessarely accurately representative of the TIME values, as the timestamp partition can be yearly/monthly... but is here to give an idea

In [6]:
%%time
aodn_dataset.get_temporal_extent()

CPU times: user 119 ms, sys: 85.2 ms, total: 204 ms
Wall time: 106 ms


(Timestamp('2017-03-16 21:11:45'), Timestamp('2023-07-29 21:41:20'))

## Read Metadata

For all Parquet datasets, we create a sidecar file named **_common_metadata** in the root of the dataset. This file contains both the dataset-level and variable-level attributes.  
The metadata can be retrieved below as a dictionary, and it will also be included in the pandas DataFrame when using the `get_data` method from the `GetAodn` class.

In [7]:
metadata = aodn_dataset.get_metadata()
metadata

2025-08-19 16:02:49,771 - aodn.GetAodn - INFO - Retrieving metadata for aodn-cloud-optimised/vessel_underway_csiro.parquet


{'rawLatitude': {'type': 'double',
  'long_name': 'Raw Latitude',
  'units': 'degree_north'},
 'latitude': {'type': 'double',
  'long_name': 'Latitude',
  'units': 'degree_north'},
 'latitudeQC': {'type': 'float'},
 'rawLongitude': {'type': 'double',
  'long_name': 'Raw Longitude',
  'units': 'degree_east'},
 'longitude': {'type': 'double',
  'long_name': 'Longitude',
  'units': 'degree_east'},
 'longitudeQC': {'type': 'float'},
 'rawAlt': {'type': 'double', 'long_name': 'Raw Altitude', 'units': 'm'},
 'alt': {'type': 'double', 'long_name': 'Altitude', 'units': 'm'},
 'altQC': {'type': 'float'},
 'rawSpeedOG': {'type': 'double',
  'long_name': 'Raw Speed Over Ground',
  'units': 'knot'},
 'speedOG': {'type': 'double',
  'long_name': 'Speed Over Ground',
  'units': 'knot'},
 'speedOGQC': {'type': 'float'},
 'rawCourseOG': {'type': 'double',
  'long_name': 'Raw Course Over Ground',
  'units': 'degree'},
 'courseOG': {'type': 'double',
  'long_name': 'Course Over Ground',
  'units': 'degr

# Data Query and Plot

## Create a TIME and BoundingBox filter

This cell loads a subset of the dataset based on a time range and a spatial bounding box. The result is returned as a pandas DataFrame, and basic information about its structure is displayed.

In [8]:
%%time
df = aodn_dataset.get_data(date_start='2017-03-16 21:11:45', 
                           date_end='2017-03-18',
                           #lat_min=-34, 
                           #lat_max=-28, 
                           #lon_min=151, 
                           #lon_max=160, 
                           )

df.info()

2025-08-19 16:02:49,893 - aodn.GetAodn - INFO - Retrieving metadata for aodn-cloud-optimised/vessel_underway_csiro.parquet


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19300 entries, 0 to 19299
Columns: 194 entries, rawLatitude to polygon
dtypes: datetime64[ns](1), float32(75), float64(113), int32(1), object(4)
memory usage: 23.0+ MB
CPU times: user 140 ms, sys: 51.5 ms, total: 192 ms
Wall time: 120 ms


In [9]:
## Download Subsetted Data as CSV

# This cell downloads the filtered dataset as a ZIP-compressed CSV file.  
# The CSV includes metadata at the top as commented lines, and a `FileLink` object is returned to allow downloading directly from the notebook.


df.aodn.download_as_csv()

/home/lbesnard/github_repo/aodn_cloud_optimised/notebooks/aodn_metadata-uuid_unknown_vessel_underway_csiro_data-hash_a55e502d.csv.zip

In [10]:
df.columns

Index(['rawLatitude', 'latitude', 'latitudeQC', 'rawLongitude', 'longitude',
       'longitudeQC', 'rawAlt', 'alt', 'altQC', 'rawSpeedOG',
       ...
       'doSaturationQC', 'rawOptodeWaterTemp', 'optodeWaterTemp',
       'optodeWaterTempQC', 'TIME', 'filename', 'ship_name', 'voyage',
       'timestamp', 'polygon'],
      dtype='object', length=194)

## Create a TIME and scalar/number filter

This cell filters the dataset by time range and a scalar value (from a Parquet partition) using the `scalar_filter` argument.  
This leverages Parquet partitioning to apply efficient, server-side filtering, which significantly speeds up data loading.

In [11]:
%%time
df = aodn_dataset.get_data(date_start='2017-03-16 21:11:45', 
                           date_end='2017-03-18',
                           scalar_filter={'voyage': 'in2017_v02'})
df.info()

2025-08-19 16:02:53,960 - aodn.GetAodn - INFO - Retrieving metadata for aodn-cloud-optimised/vessel_underway_csiro.parquet


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19300 entries, 0 to 19299
Columns: 194 entries, rawLatitude to polygon
dtypes: datetime64[ns](1), float32(75), float64(113), int32(1), object(4)
memory usage: 23.0+ MB
CPU times: user 134 ms, sys: 49 ms, total: 183 ms
Wall time: 96 ms
